In [ ]:
import pandas as pd
import os
import yaml
import requests

In [ ]:
from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/contoan/.env')

In [ ]:
ventas = pd.read_gbq("SELECT * FROM contoan.sales")

In [ ]:
#ventas.columns

In [ ]:
ventas[0<ventas['packBandana_nb']].T

In [ ]:
ventas[0<ventas['packAbanico_nb']].T

In [ ]:
print('Nº de pack bandanas {}'.format(ventas['packBandana_nb'].sum()))
print('Nº de pack abanicos {}'.format(ventas['packAbanico_nb'].sum()))
print('Dinero para Sika {}'.format((ventas['packBandana_nb']*12+ventas['packAbanico_nb']*15).sum()))

In [ ]:
5*12+4*15

## double check with shopify

In [ ]:
shop_url = os.environ['SHOPIFY_ACCESS_URL']

In [ ]:
orders = shop_url+'/orders.json'
r = requests.get(orders,
                 params={
                        'limit':250,
                        'status':'any',
                         'created_at_min':'2020-06-01T00:15:47-04:00',
                     'created_at_max':'2020-11-30T00:15:47-04:00'
                        })

In [ ]:
orders_ytd = r.json()

In [ ]:
df_orders_ytd = pd.DataFrame(orders_ytd['orders'])

In [ ]:
df_orders_ytd.shape

In [ ]:
df_orders_ytd.columns

In [ ]:
id_bandana = 4698217218147

In [ ]:
id_abanico = 4698210173027

In [ ]:
def has_bandana(x):
    for a in x:
        if a['product_id'] == id_bandana:
            return a['quantity']
        else:
            return 0

In [ ]:
def has_abanico(x):
    for a in x:
        if a['product_id'] == id_abanico:
            return a['quantity']
        else:
            return 0

In [ ]:
df_orders_ytd['has_bandana'] = df_orders_ytd.line_items.apply(lambda x: has_bandana(x))

In [ ]:
df_orders_ytd['has_bandana'].sum()

In [ ]:
df_orders_ytd[df_orders_ytd['has_bandana']==1]["created_at"]

In [ ]:
df_orders_ytd['has_abanico'] = df_orders_ytd.line_items.apply(lambda x: has_abanico(x))

In [ ]:
df_orders_ytd['has_abanico'].sum()

In [ ]:
df_orders_ytd[df_orders_ytd['has_abanico']==1]["created_at"]

In [ ]:
df_orders_ytd[(df_orders_ytd['has_bandana'] == 1) |
             (df_orders_ytd['has_abanico']==1)].to_csv('productos_con_bandana_o_abanico.csv')

In [ ]:
df_orders_ytd.line_items[0][0]['id']